# Punjabi data prep

Nay San and Prabhjot Kaur

For Punjabi, we'll use the openly available `google/fleurs` dataset from HuggingFace. A first inspection of the dataset on the web interface at https://huggingface.co/datasets/google/fleurs/viewer/pa_in/train shows there are a bunch of characters that are not from the Punjabi (Gurmukhi) script, so it may need some extra cleaning.

## Import data for analyses

In [1]:
import pandas as pd

from datasets import load_dataset
from gurmukhiutils.unicode import sanitize_unicode
from pathlib import Path

fleurs_asr = load_dataset("google/fleurs", "pa_in")

raw_data = []

for part_name, part_data in fleurs_asr.items():
    raw_data.append(
        pd.DataFrame({
            'hf_part': part_name,
            'filename': [ Path(p).name for p in part_data['path'] ],
            'dur_seconds': [ num_samples/16_000 for num_samples in part_data['num_samples'] ],
            'transcription': [ sanitize_unicode(s) for s in part_data['transcription'] ]
        })
    )
    
raw_data = pd.concat(raw_data, axis=0)

raw_data

/home/naysan/git-repos/asr-dataset-prep/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,hf_part,filename,dur_seconds,transcription
0,train,10006117397382978925.wav,10.26,ਵਿਰੋਧ ਦੇ ਆਯੋਜਕਾਂ ਨੇ ਕਿਹਾ ਕਿ ਜਰਮਨ ਦੇ ਸ਼ਹਿਰਾਂ ਜਿਵ...
1,train,1000628315543963749.wav,13.20,ਸਮਰਾਟ ਔਗਸਟਸ ਵੱਲੋਂ ਡਾਕਟਰਾਂ ਨੂੰ ਭਰਤੀ ਕਰਨਾ ਸ਼ੁਰੂ ਕ...
2,train,10038813715281955673.wav,10.86,60 ਤੋਂ ਵੱਧ ਕਰੂਜ਼ ਸਮੁੰਦਰੀ ਜਹਾਜ਼ ਗਲਾਪੈਗਸ ਪਾਣੀਆਂ ਵੱ...
3,train,10047087811721741077.wav,23.16,ਮਨੁੱਖ ਰਹਿਤ ਲਿਊਨਰ ਓਰਬਿਟਰ ਚੰਦਰਯਾਨ-1 ਨੇ ਆਪਣੇ ਮੂਨ ...
4,train,10049955308142798068.wav,8.40,"ਧਰਤੀ ‘ਤੇ ਡਿੱਗਣ ਵਾਲੇ ਤਕਰੀਬਨ 24,000 ਗਿਆਤ ਉਲਕਾ ਪਿ..."
...,...,...,...,...
569,test,9893603773578729194.wav,8.52,ਇੰਟਰਨੈੱਟ ਬਹੁ-ਗਿਣਤੀ ਅਤੇ ਅੰਤਰਵਿਅਕਤੀ ਸੰਚਾਰ ਦੋਵਾਂ ...
570,test,991393007099383171.wav,15.54,ਅਸਟ੍ਰੇਲੀਆ ਦੇ ਮਿਚਲ ਗੌਰਲੀ ਨੇ ਪੁਰਸ਼ਾਂ ਦੇ ਸਟੈਂਡਿੰਗ ...
571,test,9975828291000541250.wav,9.42,ਵਿਗਿਆਨੀ ਇਹ ਸਮਝਣ ਦੀ ਉਮੀਦ ਕਰਦੇ ਹਨ ਕਿ ਗ੍ਰਹਿ ਕਿਵੇਂ...
572,test,9985720054007195657.wav,18.90,ਕੋਕੈਮੋ ਘਾਟੀ - ਚਿੱਲੀ ਦਾ ਪ੍ਰਮੁੱਖ ਚੜ੍ਹਾਈ ਵਾਲਾ ਭਾਗ...


In [2]:
print(f"Total duration of dataset (train + val + test) in hours: {raw_data.dur_seconds.sum() / (60 * 60):.2f}")

Total duration of dataset (train + val + test) in hours: 8.95


### Show/explore raw character set

Use `CharSetExplorer` to iteratively/interactively explore the text data

In [3]:
from helpers.vocab import CharSetExplorer

raw_charset = CharSetExplorer(raw_data, 'transcription', 'filename')

100%|███████████████████████████████████████████████████████| 2748/2748 [00:00<00:00, 106710.87it/s]


In [4]:
raw_charset.print_charset()

There are 107 unique characters in the raw text (Column number of raw_chars_df: Character, Description):

(2: 'ਵ', 7715), (3: 'ਿ', 12278), (4: 'ਰ', 15272), (5: 'ੋ', 5992), (6: 'ਧ', 1341), (7: ' ', 62242), (8: 'ਦ', 10983), (9: 'ੇ', 11931), (10: 'ਆ', 5230), (11: 'ਯ', 649), (12: 'ਜ', 4580), (13: 'ਕ', 11460), (14: 'ਾ', 22114), (15: 'ਂ', 9230), (16: 'ਨ', 11077), (17: 'ਹ', 10813), (18: 'ਮ', 5168), (19: 'ਸ਼', 2488), (20: 'ਬ', 3529), (21: 'ਲ', 8012), (22: 'ੈ', 4143), (23: 'ਗ', 3609), (24: 'ਅ', 3917), (25: 'ਤ', 10314), (26: 'ੱ', 6757), (27: 'ਚ', 3788), (28: 'ਭ', 1119), (29: '1', 399), (30: '0', 572), (31: ',', 74), (32: 'ਏ', 495), (33: '।', 3029), (34: 'ਸ', 9898), (35: 'ਟ', 3030), (36: 'ਔ', 41), (37: 'ਡ', 1586), (38: 'ੂ', 3411), (39: 'ੰ', 5117), (40: 'ੀ', 12688), (41: 'ੁ', 3711), (42: 'ਣ', 2960), (43: 'ਈ', 1989), (44: 'ਪ', 5062), (45: 'ਇ', 3121), (46: '6', 127), (47: 'ਜ਼', 1142), (48: '–', 5), (49: '8', 121), (50: 'ਖ', 1655), (51: 'ਊ', 178), (52: 'ਓ', 245), (53: '-', 522), (54: '੍', 1857), (55: '

#### English characters

Looks like there's a handful of English characters 'g', 'n', etc.

In [5]:
eng_char_cols = [ col_num for (col_num, col_char) in enumerate(raw_charset.raw_chars_df.columns) if col_char in "abcdefghijklmnopqrstuvwxyz" ]

eng_char_cols

[86, 88, 89, 93, 96, 99, 100, 101, 102, 103, 104, 105, 106]

In [6]:
raw_charset.show_texts_with_chars(eng_char_cols)

,id,text,c,i,l,x,f,g,a,n,r,o,s,h,e
215,11886293421664798590.wav,ਮੌਂਟੇਵਿਡੀਓ ਸਬਟ੍ਰਾੱਪਿਕਸ ਵਿੱਚ ਸਥਿਤ ਹੈ; ਗਰਮੀਆਂ ਦੇ...,1,0,0,0,0,0,0,0,0,0,0,0,0
305,12600329852962659661.wav,1970 ਤੋਂ ਹੀ il-76 ਰੂਸੀ ਅਤੇ ਸੋਵੀਅਤ ਮਿਲਟਰੀ ਦਾ ਇੱ...,0,1,1,0,0,0,0,0,0,0,0,0,0
666,15883392946640865507.wav,ਅਜਿਹਾ ਸਾਵਧਾਨ ਯੋਜਨਾਬੰਦੀ ਨਾਲ ਆਮ ਕਾਰ ਵਿੱਚ ਕੀਤਾ ਜਾ...,0,0,0,2,0,0,0,0,0,0,0,0,0
843,17259050091492789785.wav,ਯਾਤਰੀਆਂ ਨੂੰ ਪਾਣੀ ਦਿੱਤਾ ਗਿਆ ਕਿਉਂਕਿ ਉਹਨਾਂ ਨੇ 90f...,0,0,0,0,1,0,0,0,0,0,0,0,0
2516,2958745283702930016.wav,ਜਿਆਨਕਾਰਲੋ ਫਿਸੀਸ਼ੇਲਾ giancarlo fisichella ਆਪਣੀ ਕ...,2,3,3,0,1,1,3,1,1,1,1,1,1


Looks like almost all came from 'giancarlo fisichella' in `2958745283702930016.wav`

#### Digits

Looks like there are quite a number of digits (534 rows) that'll be too much to throw away. We'll have to manually convert texts like `60 ਤੋਂ ਵੱਧ ਕਰੂਜ਼ ਸਮੁੰਦਰੀ` to `ਸੱਠ ਤੋਂ ਵੱਧ ਕਰੂਜ਼ ਸਮੁੰਦਰੀ` (assuming 'sixty' -> 'ਸੱਠ', which I used Google Translate for is correct)

In [7]:
digits_cols = [ col_num for (col_num, col_char) in enumerate(raw_charset.raw_chars_df.columns) if col_char in "1234567890" ]

digits_cols

[29, 30, 46, 49, 58, 59, 64, 65, 70, 78]

In [8]:
raw_charset.show_texts_with_chars(digits_cols)

,id,text,1,0,6,8,5,3,2,4,9,7
0,10006117397382978925.wav,ਵਿਰੋਧ ਦੇ ਆਯੋਜਕਾਂ ਨੇ ਕਿਹਾ ਕਿ ਜਰਮਨ ਦੇ ਸ਼ਹਿਰਾਂ ਜਿਵ...,1,5,0,0,0,0,0,0,0,0
2,10038813715281955673.wav,60 ਤੋਂ ਵੱਧ ਕਰੂਜ਼ ਸਮੁੰਦਰੀ ਜਹਾਜ਼ ਗਲਾਪੈਗਸ ਪਾਣੀਆਂ ਵੱ...,1,3,1,1,0,0,0,0,0,0
3,10047087811721741077.wav,ਮਨੁੱਖ ਰਹਿਤ ਲਿਊਨਰ ਓਰਬਿਟਰ ਚੰਦਰਯਾਨ-1 ਨੇ ਆਪਣੇ ਮੂਨ ...,2,3,0,0,1,1,0,0,0,0
4,10049955308142798068.wav,"ਧਰਤੀ ‘ਤੇ ਡਿੱਗਣ ਵਾਲੇ ਤਕਰੀਬਨ 24,000 ਗਿਆਤ ਉਲਕਾ ਪਿ...",0,3,0,0,0,1,1,2,0,0
8,10063369532903596472.wav,ਫਿਰ ਕੁਝ ਸਾਲਾਂ ਬਾਅਦ 1639 ਵਿੱਚ ਇੱਕ ਅੰਗਰੇਜ਼ ਖਗੋਲ ਵ...,1,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2728,944713795327015325.wav,ਇਸ ਤੋਂ ਇਲਾਵਾ ਹਰੇਕ ਰਾਜਵੰਸ਼ ਦੇ ਵਿਭਾਜਿਤ ਪ੍ਰਾਂਤਾਂ ਦ...,0,1,1,0,0,0,0,0,0,0
2729,9464068459381519511.wav,ਨੀਂਦ ਰੁਕਾਵਟ ਤੁਹਾਡੀ ਆਮ ਨੀਂਦ ਦੇ ਸਮੇਂ ਦੌਰਾਨ ਜਗਾਉਣ...,1,2,1,0,0,0,0,0,0,0
2731,9479523704585685776.wav,ਸੰਨ 1963 ਵਿੱਚ ਡੈਮ ਦੇ ਬਣਨ ਤੋਂ ਬਾਅਦ ਨਦੀ ਵਿੱਚ ਗੰਦ...,1,0,1,0,0,1,0,0,1,0
2732,9523318859027492348.wav,ਲੀਕ ਦੇ ਅਨੁਸਾਰ ਦਸਤਾਵੇਜ਼ ਹੱਦਾਂ ਦੇ ਵਿਵਾਦ ਦਾ ਉਲੇਖ ਕ...,1,0,1,0,0,0,0,0,1,1


## Process texts for ASR

In [9]:
# Manually enter this part so you can icrementally build up the expected character set

# Based on https://en.wikipedia.org/wiki/Gurmukhi_(Unicode_block)
expected_charset = set(' ' + 'ਁਂਃਅਆਇਈਉਊਏਐਓਔਕਖਗਘਙਚਛਜਝਞਟਠਡਢਣਤਥਦਧਨਪਫਬਭਮਯਰਲਲ਼ਵਸ਼ਸਹ਼ਾਿੀੁੂੇੈੋੌ੍ੑਖ਼ਗ਼ਜ਼ੜਫ਼੦੧੨੩੪੫੬੭੮੯ੰੱੲੳੴੵ')

print(sorted(list(expected_charset)))

[' ', 'ਁ', 'ਂ', 'ਃ', 'ਅ', 'ਆ', 'ਇ', 'ਈ', 'ਉ', 'ਊ', 'ਏ', 'ਐ', 'ਓ', 'ਔ', 'ਕ', 'ਖ', 'ਗ', 'ਘ', 'ਙ', 'ਚ', 'ਛ', 'ਜ', 'ਝ', 'ਞ', 'ਟ', 'ਠ', 'ਡ', 'ਢ', 'ਣ', 'ਤ', 'ਥ', 'ਦ', 'ਧ', 'ਨ', 'ਪ', 'ਫ', 'ਬ', 'ਭ', 'ਮ', 'ਯ', 'ਰ', 'ਲ', 'ਲ਼', 'ਵ', 'ਸ਼', 'ਸ', 'ਹ', '਼', 'ਾ', 'ਿ', 'ੀ', 'ੁ', 'ੂ', 'ੇ', 'ੈ', 'ੋ', 'ੌ', '੍', 'ੑ', 'ਖ਼', 'ਗ਼', 'ਜ਼', 'ੜ', 'ਫ਼', '੦', '੧', '੨', '੩', '੪', '੫', '੬', '੭', '੮', '੯', 'ੰ', 'ੱ', 'ੲ', 'ੳ', 'ੴ', 'ੵ']


In [10]:
raw_charset.print_charset(expected_charset=expected_charset)

There are 107 unique characters in the raw text (Column number of raw_chars_df: Character, Description):

(2: 'ਵ', 7715), (3: 'ਿ', 12278), (4: 'ਰ', 15272), (5: 'ੋ', 5992), (6: 'ਧ', 1341), (7: ' ', 62242), (8: 'ਦ', 10983), (9: 'ੇ', 11931), (10: 'ਆ', 5230), (11: 'ਯ', 649), (12: 'ਜ', 4580), (13: 'ਕ', 11460), (14: 'ਾ', 22114), (15: 'ਂ', 9230), (16: 'ਨ', 11077), (17: 'ਹ', 10813), (18: 'ਮ', 5168), (19: 'ਸ਼', 2488), (20: 'ਬ', 3529), (21: 'ਲ', 8012), (22: 'ੈ', 4143), (23: 'ਗ', 3609), (24: 'ਅ', 3917), (25: 'ਤ', 10314), (26: 'ੱ', 6757), (27: 'ਚ', 3788), (28: 'ਭ', 1119), (29: '1', 399), (30: '0', 572), (31: ',', 74), (32: 'ਏ', 495), (33: '।', 3029), (34: 'ਸ', 9898), (35: 'ਟ', 3030), (36: 'ਔ', 41), (37: 'ਡ', 1586), (38: 'ੂ', 3411), (39: 'ੰ', 5117), (40: 'ੀ', 12688), (41: 'ੁ', 3711), (42: 'ਣ', 2960), (43: 'ਈ', 1989), (44: 'ਪ', 5062), (45: 'ਇ', 3121), (46: '6', 127), (47: 'ਜ਼', 1142), (48: '–', 5), (49: '8', 121), (50: 'ਖ', 1655), (51: 'ਊ', 178), (52: 'ਓ', 245), (53: '-', 522), (54: '੍', 1857), (55: '

## Nay's guesses (2023-08-14):

### Digits or digit-related (to be spelled out by Prabhjot)

- Digits: (29: '1', 399), (30: '0', 572), (46: '6', 127), (49: '8', 121), (58: '5', 164), (59: '3', 114), (64: '2', 229), (65: '4', 125), (70: '9', 174), (78: '7', 91), (107: '¾', 1), (108: '½', 1) 
- Digit-related: (79: '%', 6), (97: '$', 2), (85: '°', 3),  (92: '¥', 3),

### English loanwords (remove rows, only 5 utterances)

(86: 'c', 3),  (93: 'x', 2), (89: 'l', 4), (99: 'g', 1), (100: 'a', 3), (101: 'n', 1), (102: 'r', 1), (103: 'o', 1), (104: 's', 1), (105: 'h', 1), (106: 'e', 1),  (96: 'f', 2),  (88: 'i', 4), 

### Punctuation (remove?)

(31: ',', 74), (48: '–', 5),  (53: '-', 522), (57: '.', 125),  (98: '~', 3), (80: ';', 45), (81: '?', 22), (82: '"', 32), (83: '​', 10), (84: '!', 11), (87: '—', 7), (76: ':', 49), (77: ''', 399), (90: '/', 26), (95: '’', 5),  (71: '”', 47), 

### Unknown

(33: '।', 3029), (63: '‘', 304),